# Example: build a configuration with some water molecules between two different walls

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.max_columns = None
pd.options.display.width = 100
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import MDAnalysis as mda
from MDAnalysis import transformations
import nglview as nv

from scipy.ndimage import gaussian_filter

import os,re

## build a homogenous wall

* a homogenous wall is represented by some C atoms arranged in a FCC structure with lattice constant of $5$ A, and (100) facet faces the $z$ direction.

In [2]:
from ase.build import fcc100
wall = fcc100('C', size=(20,20,5),a=5,orthogonal=True, periodic=True) # length unit: A

positions = wall.positions
positions -= np.mean(positions,axis=0) # put com at the origin
atom_num = wall.get_global_number_of_atoms()
chemical_symbols = wall.get_chemical_symbols()
edgex = wall.cell[0,0]
edgey = wall.cell[1,1]
edgez = wall.cell[2,2]
print('%d atoms with edgex=%.3f, edgex=%.3f, edgex=%.3f' %(atom_num,edgex,edgey,edgez))

view = nv.show_ase(wall)
view.clear_representations()
view.add_representation(repr_type='ball+stick',radius='0.7')
view.camera = 'orthographic'
view.add_unitcell()
#view.control.spin([1,0,0],-np.pi/2.)
view

2000 atoms with edgex=70.711, edgex=70.711, edgex=12.500


NGLWidget()

In [3]:
view.render_image(factor=8,antialias=True,trim=True,transparent=True)

Image(value=b'', width='99%')

<img title="image" alt="image" src="./images/homo_wall.png" width="400">

* save the wall in a xyz file
* ```wall.ff``` is added in the file to link the force filed file for the wall atoms
* letter 'w' added to the chemical symbols means they are *wall* atoms

In [4]:
with open('homo_wall.xyz','w') as f:
    f.write('%d\n'%atom_num)
    f.write('homo_wall wall.ff\n')
    for i in range(atom_num ):
        f.write('%s % f % f % f\n'%(chemical_symbols[i]+'w',positions[i,0],positions[i,1],positions[i,2]))

## build a heterogenous wall

* a heterogenous wall is represented by some Pt atoms at a squre center surrouned by some Si atoms, arranged in a FCC structure with lattice constant of $5$ A, and (100) facet faces the $z$ direction.

In [5]:
from ase.build import fcc100
wall = fcc100('Pt', size=(20,20,5),a=5,orthogonal=True, periodic=True) # length unit: A

positions = wall.positions
positions -= np.mean(positions,axis=0) # put com at the origin
atom_num = wall.get_global_number_of_atoms()
edgex = wall.cell[0,0]
edgey = wall.cell[1,1]
edgez = wall.cell[2,2]
print('%d atoms with edgex=%.3f, edgex=%.3f, edgex=%.3f' %(atom_num,edgex,edgey,edgez))

# change atoms at edges into another type, for example, Si
width = 15
args = np.where((positions[:,0]<-edgex/2+width) | (positions[:,0]>edgex/2-width) |
               (positions[:,1]<-edgey/2+width) | (positions[:,1]>edgey/2-width))[0] 
chemical_symbols = wall.get_chemical_symbols()
for arg in args:
    chemical_symbols[arg] = 'Si'
wall.set_chemical_symbols(chemical_symbols)

view = nv.show_ase(wall)
view.clear_representations()
view.add_representation(repr_type='ball+stick',radius='0.7')
view.camera = 'orthographic'
view.add_unitcell()
#view.control.spin([1,0,0],-np.pi/2.)
view

2000 atoms with edgex=70.711, edgex=70.711, edgex=12.500


NGLWidget()

In [6]:
view.render_image(factor=8,antialias=True,trim=True,transparent=True)

Image(value=b'', width='99%')

<img title="image" alt="image" src="./images/hetero_wall.png" width="400">

* save the wall in a xyz file
* ```wall.ff``` is added in the file to link the force filed file for the wall atoms
* letter 'w' added to the chemical symbols means they are *wall* atoms

In [7]:
with open('hetero_wall.xyz','w') as f:
    f.write('%d\n'%atom_num)
    f.write('hetero_wall wall.ff\n')
    for i in range(atom_num ):
        f.write('%s % f % f % f\n'%(chemical_symbols[i]+'w',positions[i,0],positions[i,1],positions[i,2]))

## build simulation box

* put $2000$ SPC/E water molecules between the two walls represented by the above models
* the homo. wall is at the top, and the hetero. wall is at the bottome
* initialy, the box has the same lateral sizes as the wall, and $lz=100$ in the z direction

In [8]:
water_num = 2000

lx = edgex
ly = edgey
lz = 100
distance = (lz-edgez)

* check all specis we want to put in the simulation box

In [9]:
!fftool 1 homo_wall.xyz 1 hetero_wall.xyz $water_num spce.zmat  -b $lx,$ly,$lz -c -t 2.5

density 6.649 mol/L  volume 500000.0 A^3
molecule_file      species           nmol force_field      nbond source  charge
  homo_wall.xyz    homo_wall            1 wall.ff              0 guess  +0.0000
  hetero_wall.xyz  hetero_wall          1 wall.ff              0 guess  +0.0000
  spce.zmat        SPCE              2000 spce.ff              2 file   +0.0000
packmol file
  pack.inp


* write our own ```packmol``` script with water molecules between the two walls

In [10]:
xlo = -lx/2.; ylo = -ly/2.; zlo = -lz/2.
xhi =  lx/2.; yhi =  ly/2.; zhi = lz/2.

with open('pack_mod.inp','w') as f:
    f.write('tolerance 2.0\n')
    f.write('filetype xyz\n')
    f.write('output simbox.xyz\n')
    f.write('seed -1\n')
    f.write('\n')
    
    #wall
    for i in range(2):
        if i == 0: 
            delta = -distance/2.
            wall_name = 'hetero_wall_pack'
        else: 
            delta = distance/2.
            wall_name = 'homo_wall_pack'
        f.write('structure %s.xyz\n' %wall_name)
        f.write('   number 1\n')
        f.write('   center\n')
        f.write('   fixed %f %f %f 0 0 0\n'%(0,0,delta))
        f.write('end structure\n\n')
        
    #water
    f.write('structure spce_pack.xyz\n')
    f.write('   number %d\n' %water_num)
    f.write('   inside box %f %f %f %f %f %f\n'%(xlo,ylo,zlo,xhi,yhi,zhi))
    f.write('end structure\n\n')

* run ```packmol``` to build the inital configuraiton
* ```%%capture``` stop showing output from the cell

In [11]:
%%capture
filename = './simbox.xyz'
if os.path.isfile(filename): os.remove(filename)
! packmol < pack_mod.inp

* visulize the inital simulation box

In [12]:
u = mda.Universe('simbox.xyz')
view = nv.show_mdanalysis(u)
view.clear_representations()
#view.add_unitcell()
view.add_representation(repr_type='ball+stick',radius='.7')
#view.camera = 'orthographic'
view.control.spin([1,0,0],-np.pi/2.)
view

NGLWidget()

In [13]:
view.render_image(factor=8,antialias=True,trim=True,transparent=True)

Image(value=b'', width='99%')

<img title="image" alt="image" src="./images/init_box.png" width="400">

* create ```lammps``` data file and input script by using ```fftool```

In [14]:
!fftool 1 homo_wall.xyz 1 hetero_wall.xyz $water_num spce.zmat  -b $lx,$ly,$lz -c -t 2.5 -l

density 6.649 mol/L  volume 500000.0 A^3
molecule_file      species           nmol force_field      nbond source  charge
  homo_wall.xyz    homo_wall            1 wall.ff              0 guess  +0.0000
  hetero_wall.xyz  hetero_wall          1 wall.ff              0 guess  +0.0000
  spce.zmat        SPCE              2000 spce.ff              2 file   +0.0000
lammps files units real
  in.lmp
  data.lmp
